This example aims to have two target orbits, and one observer orbit. The observer is tasked with monitoring the two targets to minimize the trace covariance of the state over the simulation horizon. We use a PPO agent that is rewarded for minimizing the trace covariance of the target states to the best of its ability.

In [1]:
%matplotlib widget
import sys
sys.path.append("../src")

import numpy as np
import matplotlib.pyplot as plt

from SensorTasking import SSA_Problem, Greedy_SSA_Problem
from plotting import visualize_info_vs_phase, render, plot_orbits, _visualize_info_other_observers_fixed

## for latin hypercube generation
from scipy.stats.qmc import LatinHypercube

In [2]:
# L2 Halo orbit, period = 3.225 TU  (Target orbit)
x0_1 = np.array([
        1.1540242813087864,
        0.0,
        -0.1384196144071876,
        4.06530060663289e-15,
        -0.21493019200956867,
        8.48098638414804e-15
    ])

# L1 lyapunov, period = 3.225 TU (target orbit)
x0_2 = np.array([
        0.8027692908754149,
        0.0,
        0.0,
        -1.1309830924549648e-14,
        0.33765564334938736,
        0.0
    ])


# 1:1 L2 Lyapunov, period = 6.45 TU (observer orbit)
x0_3 = np.array([
        0.9982702689023665,
        0.0,
        0.0,
        -2.5322340091977996e-14,
        1.5325475708886613,
        0.0
    ])

# 1:1 L1 Lyapunov, period = 6.45 TU (observer orbit)
x0_4 = np.array([
            0.65457084231188,
        0.0,
        0.0,
        3.887957091335523e-13,
        0.7413347560791179,
        0.0
])


# 2:1 Resonant, period = 6.45 TU
x0_5 = np.array([
        0.9519486347314083,
        0.0,
        0.0,
        0.0,
        -0.952445273435512,
        0.0
    ])

# 3:1 Resonant, period = 6.45 TY
x0_6 = np.array([
        0.13603399956670137,
        0.0,
        0.0,
        1.9130717669166003e-12,
        3.202418276067991,
        0.0
    ])



In [ ]:
opt = "max"
Y = np.array([x0_1, x0_2])
Y_periods = np.array([3.225, 3.225])

X = np.array([x0_4, x0_5])
X_periods = np.array([6.45, 6.45])

prob = SSA_Problem(targets=Y,
                   target_periods=Y_periods,
                   agents=X,
                   agent_periods=X_periods,
                   opt=opt)
plot_orbits(prob, fig=10, projection="xz")


In [ ]:
plt.figure(1)
# phases = np.linspace(0.385, 0.3865, 40)
phases = np.linspace(0., 1.0, 40)
visualize_info_vs_phase(prob, phases, observer=1, fig=1)
plt.tight_layout()

In [ ]:
plt.figure(2)
# phases = np.linspace(0.946, 0.951, 40)
phases = np.linspace(0., 1.0, 40)
visualize_info_vs_phase(prob, phases, observer=2, fig=2)
plt.tight_layout()

In [ ]:
_visualize_info_other_observers_fixed(prob=prob, observer = 1, fig_num = 7)

In [9]:
# Define the range for x and y
x = np.linspace(0, 0.1, 100)  
y = np.linspace(0, 0.1, 100)

# Create a meshgrid
X, Y = np.meshgrid(x, y)

Z = np.zeros_like(X)

# Iterate through each point in the meshgrid
for i in range(X.shape[0]):  # Loop over rows
    for j in range(X.shape[1]):  # Loop over columns
        candidate = [X[i, j], Y[i, j]]
        _, obj = prob.get_control_obj(candidate)
        Z[i, j] = np.log10(obj)



In [ ]:
# Create the contour plot
plt.figure(figsize=(8, 6))
contour = plt.contourf(X, Y, Z, levels=20, cmap='viridis')
cbar = plt.colorbar(contour)
cbar.set_label(r'$\log_{10}(f)$', rotation=90, labelpad=15)
plt.title('Objective Landscape')
plt.xlabel('Observer 1 Phase')
plt.ylabel('Observer 2 Phase')
plt.grid()
plt.show()

In [11]:
np.save('Z_zoom.npy', Z)